In [24]:
import os
import copy

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [16]:
class MelodyWithContextToChordSequenceDataset(Dataset):
    """
    This dataset loads the training data linking a 48 * 13 1-hot
    melody input to a 16 * 12 multi-hot chord sequence output
    """
    
    def __init__(self, data_dir, transpose=None, strip_rest_edge_measures=False):
        """
        Args:
            data_dir (string):
                directory where .txt files of training data are stored
            transpose (list[int]):
                list of semitones to transpose the data by
            strip_rest_edge_measures:
                whether or not to skip measures before the first and after the last melody note
        """
        
        self.data_dir = data_dir
        self.transpose = transpose
        self.strip_rest_edge_measures = strip_rest_edge_measures
        
        self.input_output_pairs = []
        
        for file in os.listdir(data_dir):
            melody_measures = []
            harmony_measures = []
            
            with open(os.path.join(data_dir, file)) as f_in:
                file_text = f_in.read()
                
            lines = file_text.split("\n")
            
            # Proper parsing assumes one single blank line after a measure sequence, 
            # ensure exactly one blank line at the end of the list
            while lines[-1] == "":
                lines = lines[:-1]
            
            lines.append("")
            
            song_has_melody = False
            start_measure_index = 0
            for line in lines:
                if line != "":
                    note_index = line.split()[0]
                    if note_index != "12":
                        song_has_melody = True
                        break
                else:
                    start_measure_index += 17
                        
            if song_has_melody:
                if strip_rest_edge_measures:
                    end_measure_index = len(lines)
                    for i, line in enumerate(reversed(lines)):
                        if line != "":
                            note_index = line.split()[0]
                            if note_index != "12":
                                break
                        elif i != 0 and line == "":
                            end_measure_index -= 17
                            
                    lines = lines[start_measure_index:end_measure_index]
                    
                new_melody_measure = []
                new_harmony_measure = []

                for line in lines:
                    if line == "":
                        melody_measures.append(new_melody_measure)
                        harmony_measures.append(new_harmony_measure)
                        new_melody_measure = []
                        new_harmony_measure = []
                    else:
                        melody_index, harmony_vector = line.split()
                        melody_index = int(melody_index)
                        new_melody_measure.append([1 if i == melody_index else 0 for i in range(13)])
                        new_harmony_measure.append([int(val) for val in harmony_vector])

                for i in range(len(melody_measures)):
                    if i == 0:
                        previous_melody_measure = [[0 for i in range(13)] for k in range(16)]
                    else:
                        previous_melody_measure = melody_measures[i-1]

                    if i == len(melody_measures) - 1:
                        next_melody_measure = [[0 for i in range(13)] for k in range(16)]
                    else:
                        next_melody_measure = melody_measures[i+1]

                    melody_vector = previous_melody_measure + melody_measures[i] + next_melody_measure
                    harmony_vector = harmony_measures[i]

                    melody_tensor = torch.tensor(melody_vector, dtype=torch.uint8)
                    harmony_tensor = torch.tensor(harmony_vector, dtype=torch.uint8)

                    self.input_output_pairs.append((melody_tensor, harmony_tensor))

                    if transpose:
                        for semitone_amount in transpose:
                            shifted_melody = copy.deepcopy(melody_tensor)
                            shifted_melody[:,:12] = shifted_melody[:,:12].roll(semitone_amount, dims=1)
                            shifted_harmony = copy.deepcopy(harmony_tensor).roll(semitone_amount, dims=1)

                            self.input_output_pairs.append((shifted_melody, shifted_harmony))
            else:
                print("Skipping entry with no melody")
                
    def __len__(self):
        return len(self.input_output_pairs)
    
    def __getitem__(self, idx):
        melody, harmony = self.input_output_pairs[idx]
        
        return {"melody": melody, "harmony": harmony}

In [3]:
training_data_fp = input("Input the directory where training files are stored: ")

Input the directory where training files are stored: C:\Users\Danie\PycharmProjects\ChordGenerator\data\training_data\training_dt_longnotes


In [4]:
model_path = input("Input the name of the file you would like to store the model in: ")

Input the name of the file you would like to store the model in: C:\Users\Danie\PycharmProjects\ChordGenerator\data\models\model2.pt


In [17]:
dataset = MelodyWithContextToChordSequenceDataset(
    training_data_fp, 
    transpose=[i for i in range(1, 12)], 
    strip_rest_edge_measures=True
)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

len(dataloader)

Skipping entry with no melody
Skipping entry with no melody
Skipping entry with no melody
Skipping entry with no melody
Skipping entry with no melody
Skipping entry with no melody


6759

In [20]:
positive_examples = torch.zeros([16, 12])

for datum in dataloader:
    data_batch = datum["harmony"].to(torch.float).sum(0)
    positive_examples += data_batch

negative_examples = (len(dataloader) * 32) - positive_examples

positive_weights = torch.div(negative_examples, positive_examples).flatten()

print(positive_weights)

tensor([2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372,
        2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372,
        2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372,
        2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372,
        2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372, 2.9372,
        2.9372, 2.9372, 2.9372, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369,
        2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369,
        2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369,
        2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369,
        2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369,
        2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9369, 2.9302, 2.9302, 2.9302,
        2.9302, 2.9302, 2.9302, 2.9302, 2.9302, 2.9302, 2.9302, 2.9302, 2.9302,
        2.9302, 2.9302, 2.9302, 2.9302, 

In [21]:
# Define basic prediction model
class ConditionedFeedforward(nn.Module):
    
    def __init__(self):
        super(ConditionedFeedforward, self).__init__()
        self.fc1 = nn.Linear(48 * 13, 1024)
        self.batch1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.batch2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 16 * 12)
        
    def forward(self, x):
        x = x.view(-1, 48 * 13) # Flatten input for fully connected layer
        x = self.fc1(x)
        x = self.batch1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.batch2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x
    
net = ConditionedFeedforward()

print(net)

ConditionedFeedforward(
  (fc1): Linear(in_features=624, out_features=1024, bias=True)
  (batch1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (batch2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=512, out_features=192, bias=True)
)


In [22]:
# Train prediction model

criterion = nn.BCEWithLogitsLoss(weight=positive_weights)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(250):
    
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data["melody"].to(dtype=torch.float)
        labels = data["harmony"].to(dtype=torch.float)
        labels = labels.view(-1, 16 * 12)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print('Finished Training')

[1,    50] loss: 2.030
[1,   100] loss: 1.994
[1,   150] loss: 1.960
[1,   200] loss: 1.929
[1,   250] loss: 1.900
[1,   300] loss: 1.875
[1,   350] loss: 1.850
[1,   400] loss: 1.830
[1,   450] loss: 1.811
[1,   500] loss: 1.794
[1,   550] loss: 1.775
[1,   600] loss: 1.760
[1,   650] loss: 1.743
[1,   700] loss: 1.733
[1,   750] loss: 1.720
[1,   800] loss: 1.711
[1,   850] loss: 1.698
[1,   900] loss: 1.688
[1,   950] loss: 1.682
[1,  1000] loss: 1.669
[1,  1050] loss: 1.663
[1,  1100] loss: 1.650
[1,  1150] loss: 1.646
[1,  1200] loss: 1.641
[1,  1250] loss: 1.630
[1,  1300] loss: 1.626
[1,  1350] loss: 1.620
[1,  1400] loss: 1.612
[1,  1450] loss: 1.608
[1,  1500] loss: 1.601
[1,  1550] loss: 1.597
[1,  1600] loss: 1.590
[1,  1650] loss: 1.584
[1,  1700] loss: 1.578
[1,  1750] loss: 1.578
[1,  1800] loss: 1.566
[1,  1850] loss: 1.565
[1,  1900] loss: 1.565
[1,  1950] loss: 1.562
[1,  2000] loss: 1.553
[1,  2050] loss: 1.553
[1,  2100] loss: 1.548
[1,  2150] loss: 1.546
[1,  2200] 

[3,  4400] loss: 1.284
[3,  4450] loss: 1.290
[3,  4500] loss: 1.294
[3,  4550] loss: 1.304
[3,  4600] loss: 1.319
[3,  4650] loss: 1.290
[3,  4700] loss: 1.283
[3,  4750] loss: 1.292
[3,  4800] loss: 1.309
[3,  4850] loss: 1.299
[3,  4900] loss: 1.298
[3,  4950] loss: 1.322
[3,  5000] loss: 1.322
[3,  5050] loss: 1.295
[3,  5100] loss: 1.286
[3,  5150] loss: 1.290
[3,  5200] loss: 1.295
[3,  5250] loss: 1.292
[3,  5300] loss: 1.300
[3,  5350] loss: 1.302
[3,  5400] loss: 1.299
[3,  5450] loss: 1.291
[3,  5500] loss: 1.321
[3,  5550] loss: 1.304
[3,  5600] loss: 1.302
[3,  5650] loss: 1.300
[3,  5700] loss: 1.297
[3,  5750] loss: 1.289
[3,  5800] loss: 1.302
[3,  5850] loss: 1.305
[3,  5900] loss: 1.299
[3,  5950] loss: 1.300
[3,  6000] loss: 1.297
[3,  6050] loss: 1.281
[3,  6100] loss: 1.297
[3,  6150] loss: 1.297
[3,  6200] loss: 1.302
[3,  6250] loss: 1.289
[3,  6300] loss: 1.296
[3,  6350] loss: 1.300
[3,  6400] loss: 1.294
[3,  6450] loss: 1.301
[3,  6500] loss: 1.279
[3,  6550] 

[6,  2000] loss: 1.219
[6,  2050] loss: 1.248
[6,  2100] loss: 1.262
[6,  2150] loss: 1.241
[6,  2200] loss: 1.234
[6,  2250] loss: 1.228
[6,  2300] loss: 1.252
[6,  2350] loss: 1.234
[6,  2400] loss: 1.241
[6,  2450] loss: 1.244
[6,  2500] loss: 1.231
[6,  2550] loss: 1.230
[6,  2600] loss: 1.233
[6,  2650] loss: 1.247
[6,  2700] loss: 1.256
[6,  2750] loss: 1.242
[6,  2800] loss: 1.247
[6,  2850] loss: 1.228
[6,  2900] loss: 1.229
[6,  2950] loss: 1.246
[6,  3000] loss: 1.247
[6,  3050] loss: 1.244
[6,  3100] loss: 1.242
[6,  3150] loss: 1.249
[6,  3200] loss: 1.229
[6,  3250] loss: 1.240
[6,  3300] loss: 1.228
[6,  3350] loss: 1.247
[6,  3400] loss: 1.245
[6,  3450] loss: 1.251
[6,  3500] loss: 1.254
[6,  3550] loss: 1.225
[6,  3600] loss: 1.235
[6,  3650] loss: 1.224
[6,  3700] loss: 1.251
[6,  3750] loss: 1.242
[6,  3800] loss: 1.266
[6,  3850] loss: 1.252
[6,  3900] loss: 1.253
[6,  3950] loss: 1.245
[6,  4000] loss: 1.245
[6,  4050] loss: 1.244
[6,  4100] loss: 1.211
[6,  4150] 

[8,  6350] loss: 1.194
[8,  6400] loss: 1.210
[8,  6450] loss: 1.206
[8,  6500] loss: 1.224
[8,  6550] loss: 1.206
[8,  6600] loss: 1.189
[8,  6650] loss: 1.193
[8,  6700] loss: 1.210
[8,  6750] loss: 1.207
[9,    50] loss: 1.185
[9,   100] loss: 1.174
[9,   150] loss: 1.189
[9,   200] loss: 1.175
[9,   250] loss: 1.214
[9,   300] loss: 1.195
[9,   350] loss: 1.176
[9,   400] loss: 1.199
[9,   450] loss: 1.184
[9,   500] loss: 1.144
[9,   550] loss: 1.187
[9,   600] loss: 1.178
[9,   650] loss: 1.188
[9,   700] loss: 1.190
[9,   750] loss: 1.204
[9,   800] loss: 1.186
[9,   850] loss: 1.193
[9,   900] loss: 1.181
[9,   950] loss: 1.195
[9,  1000] loss: 1.205
[9,  1050] loss: 1.184
[9,  1100] loss: 1.192
[9,  1150] loss: 1.178
[9,  1200] loss: 1.190
[9,  1250] loss: 1.186
[9,  1300] loss: 1.170
[9,  1350] loss: 1.210
[9,  1400] loss: 1.177
[9,  1450] loss: 1.159
[9,  1500] loss: 1.177
[9,  1550] loss: 1.176
[9,  1600] loss: 1.191
[9,  1650] loss: 1.183
[9,  1700] loss: 1.187
[9,  1750] 

[11,  3500] loss: 1.125
[11,  3550] loss: 1.147
[11,  3600] loss: 1.146
[11,  3650] loss: 1.159
[11,  3700] loss: 1.140
[11,  3750] loss: 1.140
[11,  3800] loss: 1.143
[11,  3850] loss: 1.145
[11,  3900] loss: 1.155
[11,  3950] loss: 1.167
[11,  4000] loss: 1.149
[11,  4050] loss: 1.118
[11,  4100] loss: 1.150
[11,  4150] loss: 1.176
[11,  4200] loss: 1.155
[11,  4250] loss: 1.154
[11,  4300] loss: 1.148
[11,  4350] loss: 1.144
[11,  4400] loss: 1.141
[11,  4450] loss: 1.150
[11,  4500] loss: 1.152
[11,  4550] loss: 1.162
[11,  4600] loss: 1.148
[11,  4650] loss: 1.144
[11,  4700] loss: 1.137
[11,  4750] loss: 1.139
[11,  4800] loss: 1.136
[11,  4850] loss: 1.157
[11,  4900] loss: 1.156
[11,  4950] loss: 1.146
[11,  5000] loss: 1.121
[11,  5050] loss: 1.165
[11,  5100] loss: 1.167
[11,  5150] loss: 1.139
[11,  5200] loss: 1.164
[11,  5250] loss: 1.164
[11,  5300] loss: 1.130
[11,  5350] loss: 1.147
[11,  5400] loss: 1.163
[11,  5450] loss: 1.152
[11,  5500] loss: 1.119
[11,  5550] loss

...

[241,   650] loss: 0.516
[241,   700] loss: 0.538
[241,   750] loss: 0.551
[241,   800] loss: 0.535
[241,   850] loss: 0.526
[241,   900] loss: 0.537
[241,   950] loss: 0.531
[241,  1000] loss: 0.547
[241,  1050] loss: 0.515
[241,  1100] loss: 0.542
[241,  1150] loss: 0.549
[241,  1200] loss: 0.550
[241,  1250] loss: 0.551
[241,  1300] loss: 0.537
[241,  1350] loss: 0.508
[241,  1400] loss: 0.526
[241,  1450] loss: 0.509
[241,  1500] loss: 0.530
[241,  1550] loss: 0.547
[241,  1600] loss: 0.537
[241,  1650] loss: 0.540
[241,  1700] loss: 0.517
[241,  1750] loss: 0.522
[241,  1800] loss: 0.545
[241,  1850] loss: 0.525
[241,  1900] loss: 0.532
[241,  1950] loss: 0.538
[241,  2000] loss: 0.554
[241,  2050] loss: 0.557
[241,  2100] loss: 0.528
[241,  2150] loss: 0.553
[241,  2200] loss: 0.539
[241,  2250] loss: 0.523
[241,  2300] loss: 0.533
[241,  2350] loss: 0.530
[241,  2400] loss: 0.520
[241,  2450] loss: 0.533
[241,  2500] loss: 0.512
[241,  2550] loss: 0.538
[241,  2600] loss: 0.564


[243,  3550] loss: 0.540
[243,  3600] loss: 0.536
[243,  3650] loss: 0.536
[243,  3700] loss: 0.541
[243,  3750] loss: 0.566
[243,  3800] loss: 0.543
[243,  3850] loss: 0.542
[243,  3900] loss: 0.557
[243,  3950] loss: 0.526
[243,  4000] loss: 0.547
[243,  4050] loss: 0.521
[243,  4100] loss: 0.533
[243,  4150] loss: 0.528
[243,  4200] loss: 0.558
[243,  4250] loss: 0.562
[243,  4300] loss: 0.545
[243,  4350] loss: 0.542
[243,  4400] loss: 0.532
[243,  4450] loss: 0.531
[243,  4500] loss: 0.541
[243,  4550] loss: 0.528
[243,  4600] loss: 0.563
[243,  4650] loss: 0.563
[243,  4700] loss: 0.561
[243,  4750] loss: 0.518
[243,  4800] loss: 0.549
[243,  4850] loss: 0.537
[243,  4900] loss: 0.553
[243,  4950] loss: 0.532
[243,  5000] loss: 0.536
[243,  5050] loss: 0.550
[243,  5100] loss: 0.533
[243,  5150] loss: 0.552
[243,  5200] loss: 0.531
[243,  5250] loss: 0.522
[243,  5300] loss: 0.537
[243,  5350] loss: 0.541
[243,  5400] loss: 0.535
[243,  5450] loss: 0.548
[243,  5500] loss: 0.521


[245,  6450] loss: 0.545
[245,  6500] loss: 0.542
[245,  6550] loss: 0.551
[245,  6600] loss: 0.545
[245,  6650] loss: 0.548
[245,  6700] loss: 0.534
[245,  6750] loss: 0.551
[246,    50] loss: 0.508
[246,   100] loss: 0.519
[246,   150] loss: 0.518
[246,   200] loss: 0.519
[246,   250] loss: 0.518
[246,   300] loss: 0.520
[246,   350] loss: 0.497
[246,   400] loss: 0.524
[246,   450] loss: 0.522
[246,   500] loss: 0.506
[246,   550] loss: 0.517
[246,   600] loss: 0.533
[246,   650] loss: 0.528
[246,   700] loss: 0.506
[246,   750] loss: 0.511
[246,   800] loss: 0.555
[246,   850] loss: 0.504
[246,   900] loss: 0.529
[246,   950] loss: 0.518
[246,  1000] loss: 0.539
[246,  1050] loss: 0.524
[246,  1100] loss: 0.542
[246,  1150] loss: 0.534
[246,  1200] loss: 0.548
[246,  1250] loss: 0.552
[246,  1300] loss: 0.530
[246,  1350] loss: 0.523
[246,  1400] loss: 0.521
[246,  1450] loss: 0.527
[246,  1500] loss: 0.534
[246,  1550] loss: 0.533
[246,  1600] loss: 0.544
[246,  1650] loss: 0.584


[248,  2600] loss: 0.525
[248,  2650] loss: 0.530
[248,  2700] loss: 0.544
[248,  2750] loss: 0.522
[248,  2800] loss: 0.537
[248,  2850] loss: 0.518
[248,  2900] loss: 0.554
[248,  2950] loss: 0.533
[248,  3000] loss: 0.533
[248,  3050] loss: 0.522
[248,  3100] loss: 0.520
[248,  3150] loss: 0.525
[248,  3200] loss: 0.527
[248,  3250] loss: 0.538
[248,  3300] loss: 0.567
[248,  3350] loss: 0.537
[248,  3400] loss: 0.543
[248,  3450] loss: 0.542
[248,  3500] loss: 0.540
[248,  3550] loss: 0.546
[248,  3600] loss: 0.549
[248,  3650] loss: 0.523
[248,  3700] loss: 0.532
[248,  3750] loss: 0.541
[248,  3800] loss: 0.542
[248,  3850] loss: 0.546
[248,  3900] loss: 0.543
[248,  3950] loss: 0.553
[248,  4000] loss: 0.546
[248,  4050] loss: 0.516
[248,  4100] loss: 0.540
[248,  4150] loss: 0.536
[248,  4200] loss: 0.558
[248,  4250] loss: 0.507
[248,  4300] loss: 0.538
[248,  4350] loss: 0.502
[248,  4400] loss: 0.548
[248,  4450] loss: 0.521
[248,  4500] loss: 0.521
[248,  4550] loss: 0.551


[250,  5500] loss: 0.516
[250,  5550] loss: 0.529
[250,  5600] loss: 0.537
[250,  5650] loss: 0.562
[250,  5700] loss: 0.533
[250,  5750] loss: 0.530
[250,  5800] loss: 0.538
[250,  5850] loss: 0.539
[250,  5900] loss: 0.558
[250,  5950] loss: 0.543
[250,  6000] loss: 0.543
[250,  6050] loss: 0.533
[250,  6100] loss: 0.518
[250,  6150] loss: 0.496
[250,  6200] loss: 0.551
[250,  6250] loss: 0.530
[250,  6300] loss: 0.527
[250,  6350] loss: 0.530
[250,  6400] loss: 0.550
[250,  6450] loss: 0.523
[250,  6500] loss: 0.546
[250,  6550] loss: 0.559
[250,  6600] loss: 0.509
[250,  6650] loss: 0.532
[250,  6700] loss: 0.517
[250,  6750] loss: 0.507
Finished Training


In [23]:
torch.save(net.state_dict(), model_path)